In [53]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report 
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from hyperopt import hp 
from hyperopt import fmin, tpe, Trials 
from sklearn.ensemble import GradientBoostingClassifier

In [108]:
pca_train = pd.read_csv('PCA_train_data.csv')
pca_test = pd.read_csv('PCA_test_data.csv')
tree_train = pd.read_csv('Tree_train_data.csv')
tree_test= pd.read_csv('Tree_test_data.csv')

In [50]:
pca_train.head()


,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,attack,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,...,is_attack,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9
0,0,0,0,0,0,0,normal,0.0,1.0,0.0,...,0.0,-0.940599,-0.473843,0.719614,0.117456,-0.357645,-0.162415,-0.191790,-0.219253,-0.014388
1,0,0,0,0,0,0,normal,0.0,0.0,1.0,...,0.0,-0.728731,-0.240017,1.426927,0.096360,0.159278,-0.026683,0.227241,0.402077,-0.355985
2,0,0,0,0,0,0,neptune,0.0,1.0,0.0,...,1.0,3.035355,0.142109,-0.102676,-0.026129,-0.011964,0.012623,-0.021454,0.066945,-0.089175
3,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.0,-0.802288,-0.822607,-0.560944,0.151033,-0.207357,-0.232259,0.136882,0.050881,0.049239
4,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.0,-1.134026,-0.812825,-0.190227,-0.525232,0.102108,0.061269,-0.123393,0.092231,-0.040100


In [109]:
X_pca_train = pca_train.drop(columns=['is_attack', 'attack_category',"attack"])
y_pca_train = pca_train["attack_category"]
X_pca_test = pca_test.drop(columns=['is_attack', 'attack_category',"attack"])
y_pca_test= pca_test["attack_category"]


In [110]:
X_tree_train = tree_train.drop(columns=['is_attack', 'attack_category',"attack"])
y_tree_train = tree_train["attack_category"]
X_tree_test = tree_test.drop(columns=['is_attack', 'attack_category',"attack"])
y_tree_test= tree_test["attack_category"]

In [111]:
X_tree_train

,logged_in,root_shell,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,...,diff_srv_rate_scaled,srv_diff_host_rate_scaled,dst_host_count_scaled,dst_host_srv_count_scaled,dst_host_same_srv_rate_scaled,dst_host_diff_srv_rate_scaled,dst_host_same_src_port_rate_scaled,dst_host_srv_diff_host_rate_scaled,serror_rate_combined,rerror_rate_combined
0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.05
1,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00
2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,4.00,0.00
3,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.44,0.01
4,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,4.00,0.00
125969,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00
125970,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.01
125971,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.05,0.00,1.000000,0.031373,0.03,0.05,0.00,0.00,4.00,0.00


In [112]:
X_pca_train

,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_OTH,...,flag_SH,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9
0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.940599,-0.473843,0.719614,0.117456,-0.357645,-0.162415,-0.191790,-0.219253,-0.014388
1,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-0.728731,-0.240017,1.426927,0.096360,0.159278,-0.026683,0.227241,0.402077,-0.355985
2,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.035355,0.142109,-0.102676,-0.026129,-0.011964,0.012623,-0.021454,0.066945,-0.089175
3,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.802288,-0.822607,-0.560944,0.151033,-0.207357,-0.232259,0.136882,0.050881,0.049239
4,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-1.134026,-0.812825,-0.190227,-0.525232,0.102108,0.061269,-0.123393,0.092231,-0.040100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.026111,0.134002,-0.102845,-0.034096,0.043781,-0.004933,-0.007050,-0.006698,0.012918
125969,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-1.119040,-0.792833,-0.115345,-0.513639,0.062364,-0.016443,-0.155788,0.095988,-0.082379
125970,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.211762,-0.391754,0.632769,-0.190737,-0.261550,-0.086665,-0.416727,-0.160747,0.057605
125971,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.059266,0.171533,0.003287,-0.001806,-0.018560,0.020528,-0.023829,-0.019866,-0.062215


In [94]:
def train_model(model, X, y):
    skf = StratifiedKFold(n_splits=5)

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for train_index, test_index in skf.split(X, y):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        model.fit(X_train, y_train)

        pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        precision = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted') 
        f1 = f1_score(y_test, pred, average='weighted') 

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    # print('{0} Accuracy: {1: .5f} Precision: {2: .5f} Recall: {3: .5f} F1_Score: {4: .5f}'.format(
    #     model.__class__.__name__, 
    #     np.mean(accuracy_scores),
    #     np.mean(precision_scores),
    #     np.mean(recall_scores),
    #     np.mean(f1_scores)))
    
    return -np.mean(f1_scores)

In [95]:
def evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print("Model: ", model.__class__.__name__)
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("####################################")
    
    # report = classification_report(y_test, y_pred)
    # print("Classification Report:\n", report)
    
    return -f1

In [122]:
def best_params(objective_func, search_space):
    trials = Trials() 

    best = fmin(fn=objective_func,
                space=search_space,
                algo=tpe.suggest,
                max_evals=50,
                trials=trials,
                rstate=np.random.default_rng(seed=30))

    print('Best Parameters:', best)
    return best 

In [ ]:
clf =RandomForestClassifier(n_estimators=100, random_state=0)
evaluate(clf, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

In [72]:
rf_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 5, 30, 5),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
}

def objective_func_rf(search_space):
    # Create and train a Random Forest classifier with the specified hyperparameters
    rf_clf = RandomForestClassifier(
        n_estimators=int(search_space['n_estimators']),
        max_depth=int(search_space['max_depth']),
        min_samples_split=search_space['min_samples_split'],
        min_samples_leaf=search_space['min_samples_leaf'],
        random_state=0
    )

 
    return train_model(rf_clf,X_pca_train,y_pca_train)
    
    
    
    


In [73]:
rf_best = best_params(objective_func_rf, rf_search_space)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [20:59<00:00, 25.19s/trial, best loss: -0.8049732468120633]
Beat Parameters: {'max_depth': 15.0, 'min_samples_leaf': 0.2513793095504232, 'min_samples_split': 0.3133569525978853, 'n_estimators': 400.0}


In [75]:
clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=15.0,
    min_samples_split=0.3133569525978853,
    min_samples_leaf=0.2513793095504232,
    random_state=0 
)
evaluate(clf, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  RandomForestClassifier
Accuracy:  0.646735273243435
Precision:  0.4984827668513077
Recall:  0.646735273243435
F1 Score:  0.5552161747350216
####################################


-0.5552161747350216

For Tree Feature Selection

In [77]:
clf =RandomForestClassifier(n_estimators=100, random_state=0)
evaluate( clf, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  RandomForestClassifier
Accuracy:  0.7496451383960255
Precision:  0.8162980247573882
Recall:  0.7496451383960255
F1 Score:  0.708170542866704
####################################


-0.708170542866704

In [78]:
rf_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 5, 30, 5),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
}

def objective_func_rf2(search_space):
    # Create and train a Random Forest classifier with the specified hyperparameters
    rf_clf = RandomForestClassifier(
        n_estimators=int(search_space['n_estimators']),
        max_depth=int(search_space['max_depth']),
        min_samples_split=search_space['min_samples_split'],
        min_samples_leaf=search_space['min_samples_leaf'],
        random_state=0
    )

 
    return train_model(rf_clf,X_tree_train,y_tree_train)


    

In [79]:
rf2_best = best_params(objective_func_rf2, rf_search_space)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [56:28<00:00, 67.76s/trial, best loss: -0.8133003608486226]   
Beat Parameters: {'max_depth': 5.0, 'min_samples_leaf': 0.24992729271137337, 'min_samples_split': 0.10955712733391201, 'n_estimators': 400.0}


In [ ]:
print("Best hyperparameters:")
print(best)

In [81]:
clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=5.0,
    min_samples_split=0.10955712733391201,
    min_samples_leaf=0.24992729271137337,
    random_state=0 
)
evaluate( clf, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  RandomForestClassifier
Accuracy:  0.6307665010645848
Precision:  0.4952307529492295
Recall:  0.6307665010645848
F1 Score:  0.5410100800055753
####################################


-0.5410100800055753

GBM

In [98]:
X_pca_train

,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_OTH,...,flag_SH,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9
0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.940599,-0.473843,0.719614,0.117456,-0.357645,-0.162415,-0.191790,-0.219253,-0.014388
1,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-0.728731,-0.240017,1.426927,0.096360,0.159278,-0.026683,0.227241,0.402077,-0.355985
2,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.035355,0.142109,-0.102676,-0.026129,-0.011964,0.012623,-0.021454,0.066945,-0.089175
3,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.802288,-0.822607,-0.560944,0.151033,-0.207357,-0.232259,0.136882,0.050881,0.049239
4,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-1.134026,-0.812825,-0.190227,-0.525232,0.102108,0.061269,-0.123393,0.092231,-0.040100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.026111,0.134002,-0.102845,-0.034096,0.043781,-0.004933,-0.007050,-0.006698,0.012918
125969,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-1.119040,-0.792833,-0.115345,-0.513639,0.062364,-0.016443,-0.155788,0.095988,-0.082379
125970,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.211762,-0.391754,0.632769,-0.190737,-0.261550,-0.086665,-0.416727,-0.160747,0.057605
125971,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.059266,0.171533,0.003287,-0.001806,-0.018560,0.020528,-0.023829,-0.019866,-0.062215


In [99]:
X_tree_train

,logged_in,root_shell,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,...,diff_srv_rate_scaled,srv_diff_host_rate_scaled,dst_host_count_scaled,dst_host_srv_count_scaled,dst_host_same_srv_rate_scaled,dst_host_diff_srv_rate_scaled,dst_host_same_src_port_rate_scaled,dst_host_srv_diff_host_rate_scaled,serror_rate_combined,rerror_rate_combined
0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.05
1,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00
2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,4.00,0.00
3,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.44,0.01
4,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,4.00,0.00
125969,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00
125970,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.01
125971,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.05,0.00,1.000000,0.031373,0.03,0.05,0.00,0.00,4.00,0.00


In [118]:
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
evaluate(gbm, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7806068133427964
Precision:  0.8276618375114969
Recall:  0.7806068133427964
F1 Score:  0.7569507339870514
####################################


-0.7569507339870514

In [119]:
# Define the search space for hyperparameters
gbm_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 1, 20, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
    'subsample': hp.uniform('subsample', 0.1, 1.0),
}

# Define the objective function for optimizing F1 score
def objective_func_gbm(params):
    gbm = GradientBoostingClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        subsample=params['subsample'],
        random_state=0
    )
    

    return train_model(gbm,X_pca_train,y_pca_train)



In [120]:
gbm_best = best_params(objective_func_gbm, gbm_search_space)

100%|██████████| 50/50 [11:07:15<00:00, 800.71s/trial, best loss: -0.9926813509759158]   
Best Parameters: {'learning_rate': 0.37121026674940893, 'max_depth': 8.0, 'min_samples_leaf': 0.10054390556897791, 'min_samples_split': 0.16856156690516794, 'n_estimators': 450.0, 'subsample': 0.8996148153668627}


In [126]:
gbm = GradientBoostingClassifier(n_estimators=450, learning_rate= 0.37121026674940893, max_depth=8.0,min_samples_leaf=0.10054390556897791, min_samples_split= 0.16856156690516794, subsample=0.8996148153668627, random_state=0)
evaluate(gbm, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7537704045422285
Precision:  0.8108858414796004
Recall:  0.7537704045422285
F1 Score:  0.7209796253897186
####################################


-0.7209796253897186

In [121]:
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
evaluate(gbm, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7488023420865862
Precision:  0.8113205003485532
Recall:  0.7488023420865862
F1 Score:  0.7036452715792411
####################################


-0.7036452715792411

In [127]:
# Define the search space for hyperparameters
gbm_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 1, 20, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
    'subsample': hp.uniform('subsample', 0.1, 1.0),
}

# Define the objective function for optimizing F1 score
def objective_func_gbm2(params):
    gbm = GradientBoostingClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        subsample=params['subsample'],
        random_state=0
    )
    

    return train_model(gbm,X_tree_train,y_tree_train)


In [128]:
gbm2_best = best_params(objective_func_gbm2, gbm_search_space)

100%|██████████| 50/50 [4:52:47<00:00, 351.34s/trial, best loss: -0.9895762709813092]  
Best Parameters: {'learning_rate': 0.03523010317024397, 'max_depth': 10.0, 'min_samples_leaf': 0.10046592387004868, 'min_samples_split': 0.10655581077489396, 'n_estimators': 350.0, 'subsample': 0.9865122012440967}


In [129]:
gbm = GradientBoostingClassifier(n_estimators=350, learning_rate= 0.03523010317024397, max_depth=10.0,min_samples_leaf=0.10046592387004868, min_samples_split= 0.10655581077489396, subsample=0.9865122012440967, random_state=0)
evaluate(gbm, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7341199432221434
Precision:  0.796853127179646
Recall:  0.7341199432221434
F1 Score:  0.6930681907940133
####################################


-0.6930681907940133